In [1]:
from utils.config import simulations_path, potentials_path, potentials, experiments_path, executables_path, executable, atom_type1, atom_type2
from utils.default_dictionaries import base_params_atoms_position, defect_percentages, reduction_percentages,gen_lammps_input,vacancy_percentages  
from utils.write_input_from_default import generate_experiments_jsons_files, write_lammps_input
from utils.gen_defect_sim import generate_atom_type_array, reduce_concentration, generate_defect_positions,introduce_vacancies
from utils.gen_positions import generate_atomic_positions
from utils.gen_atom_pos_file import write_atoms_pos_file
from utils.default_dict_sbatch import slurm_job_sbatch_dict
from utils.write_sbatch import write_sbatch
import numpy as np
import subprocess
import shutil
import os
import json

In [2]:
#######################################################################
############################ LOCAL MACHINE ############################
#######################################################################
# -------- Uncomment the following lines to run UTEP in a Local Machine
# -------- If you will run in a supercomputer, skip this cell
# !git clone https://github.com/lammps/lammps.git lammps
# %cd ./lammps/
# !rm -rf ./lammps/build
# !mkdir build
# %cd build
# !cmake ../cmake -DLAMMPS_EXCEPTIONS=yes \
#                -DBUILD_SHARED_LIBS=yes \
#                -DMLIAP_ENABLE_PYTHON=yes \
#                -DPYTHON_EXECUTABLE:FILEPATH=`which python`
#                -DPKG_MPI=ON

# !make -j 2
# !make install-python
# lmp = lammps.lammps()
# print(lmp)


In [2]:
# -------- First step create the experiments parameter for each simulation in json files --------
base_params = base_params_atoms_position
generate_experiments_jsons_files(base_params, experiments_path, mode="vacancy", vacancy_percentages=vacancy_percentages, reduction_percentages=reduction_percentages)

Created ./Experiments_vacancy/simulation_conc0.50-0.50_vacancy_0.1%.json
Created ./Experiments_vacancy/simulation_conc0.50-0.50_vacancy_0.5%.json
Created ./Experiments_vacancy/simulation_conc0.50-0.50_vacancy_1%.json


In [ ]:
# -------- Utilities To Execute Pipelines (TEDEHAMO)
'''

This code begins loading the JSON files with the initial paramaterers for every simulation, 
from util.default_dictionaries you can edit with your own parameters as well that you decide which point 
defect will be used (vacancy or antisite) 

'''

for conc in reduction_percentages: # per concentration reduction
    concentration_path_sim = simulations_path + f'{(0.50-conc):.2f}-{atom_type1}-{(0.50+conc):.2f}-{atom_type2}/'
    os.makedirs(concentration_path_sim, exist_ok=True)
    for vacancy in vacancy_percentages: # for vacancy change to vacancy_percentages
        vacancy_path_sim = concentration_path_sim + f'vacancy_{vacancy}%'
        os.makedirs(vacancy_path_sim, exist_ok=True)
        # --------- loading the json files
        input_path_json = experiments_path +f'simulation_conc{(0.50-conc):.2f}-{(0.50+conc):.2f}_vacancy_{vacancy}%.json'
        with open(input_path_json, 'r') as f:
            params = json.load(f)
        
        # --------- Extracting parameters from the JSON
        initial_temperature = params['initial_temperature']
        initial_lattice_parameter = params['initial_lattice_parameter']
        crystal_structure = params['crystal_structure']
        system_size = params['system_size']
        vacancy_percentage = params['vacancy_percentage']
        reduction_percentage  = params['reduction_percentage']
        if reduction_percentage >= 0:
            atom_type_to_reduce = params['index_atom_type_2']
        else:
            atom_type_to_reduce = params['index_atom_type_1']
        print(conc,atom_type_to_reduce)
        n_temp_variations = params['n_temp_variations']
        m_lattice_variations = params['m_lattice_variations']
        # atom_type_to_remove = params["atom_type_to_remove"] #atom to be removed (vacancy)
        # vacancy_percentage = params["vacancy_percentage"]
        
        # ---------- Data Generation 
        lattice_parameters = [initial_lattice_parameter + i * 0.01 for i in range(m_lattice_variations)]
        total_atoms = 2 * (system_size ** 3)
        atom_type_array = np.zeros(total_atoms)
        initial_b2_atom_array, type_1,type_2 = generate_atom_type_array(total_atoms)
        # print(atom_type_to_reduce, reduction_percentage)
        b2_atom_array, type_1, type_2  = reduce_concentration(initial_b2_atom_array, atom_type_to_reduce, reduction_percentage) #1 = Ni, 2 = Ti
        print(reduction_percentage,type_1, type_2)
        # print(type_1,type_2,b2_atom_array[0:10])
        b2_atom_array_swapped, type_1, type_2,indices_selected_vacancy =introduce_vacancies(b2_atom_array,1,vacancy) #atom_type_array, atom_type_to_remove, vacancy_percentageintroduce_vacancies(total_atoms, b2_atom_array, vacancy_percentage)
        # b2_atom_array_swapped, type_1, type_2 = introduce_vacancies(b2_atom_array, atom_type_to_remove, vacancy_percentage)
        print('------',type_1,type_2)

        # print(f'{50-conc}-{50+conc}',defect,atom_selected_type_1, atom_selected_type_2,b2_atom_array_swapped[0:20],type_1, type_2)
        summary_file = os.path.join(experiments_path, 'simulation_summary.txt')
        
        for lattice_parameter in lattice_parameters:
            basis = np.array([[1, 0.0, 0.0],
                              [0.0, 1, 0.0],
                              [0.0, 0.0, 1]]) * lattice_parameter
    
            base_atoms = np.array([[0.0, 0.0, 0.0],
                                   [0.5, 0.5, 0.5]]) * lattice_parameter
    
        # ---------- Generate atom positions
            positions = []
            for i in range(system_size):
                for j in range(system_size):
                    for k in range(system_size):
                        base_position = np.array([i, j, k])
                        cart_position = np.dot(basis.T, base_position)
                        for atom in base_atoms:
                            positions.append(cart_position + atom)
            # ---------- In vacancy defect uncomment the following lines to identify the vacancy sites.
            positions = [pos for ii, pos in enumerate(positions) if ii not in indices_selected_vacancy]
            vacancies = [pos for ii, pos in enumerate(positions) if ii in indices_selected_vacancy]
                
        # ---------- Generate the temperature loop   
            for temperature_variation in range(n_temp_variations):
                temperature = temperature_variation * 100 + initial_temperature # ---------- temp incresead by 100 degrees across ntemp iterations

                directory = vacancy_path_sim + f'/{atom_type1}{atom_type2}_T_{int(temperature)}_L_{lattice_parameter:.2f}'
                os.makedirs(directory, exist_ok=True)
        
                gen_lammps_input['md_settings']['fix']['temp'] = [temperature, temperature, 0.01]
                gen_lammps_input['md_settings']['velocity']['all']['create'][0] = temperature
                input_file_name = f'{atom_type1}{atom_type2}_Temp_{int(temperature)}_Latt_{lattice_parameter:.2f}.in'
                dir_sim = os.path.join(directory, input_file_name)
                write_lammps_input(gen_lammps_input, dir_sim)
        # ---------- Copy and paste potentials to each simulation  
                for pot_file in potentials:
                    src = os.path.join(potentials_path, pot_file)
                    dest = os.path.join(directory, pot_file)
                    shutil.copy(src, dest)
                shutil.copy(executables_path+executable,directory)
                atoms_pos_paths = os.path.join(directory, 'atoms_positions.data')
        # ---------- Create atom_positions.data with points defects added  
                write_atoms_pos_file(atoms_pos_paths, lattice_parameter, type_1/(type_1+type_2), type_2/(type_1+type_2), positions, b2_atom_array_swapped, system_size,vacancy,atom_type1,atom_type2)
        
        # ---------- Write and copy to each directory simulation the job script, can be edited in utils.default_dict_sbatch
                write_sbatch(directory,input_file_name,slurm_job_sbatch_dict)
                subprocess.run(['sbatch', 'run.sbatch'], cwd=directory)

        # ---------- If you want to performe those simulations in a local machine, comment the last step and
        # ---------- uncomment the following line and the cell labeled as LOCAL MACHINE
        #subprocess.run(['mpirun -np 4 lmp -in', 'input_file_name'], cwd=directory)


            

0 2.0
No atoms changed due to small reduction_percentage.
0 1000 1000
1
[   0    2    4    6    8   10   12   14   16   18   20   22   24   26
   28   30   32   34   36   38   40   42   44   46   48   50   52   54
   56   58   60   62   64   66   68   70   72   74   76   78   80   82
   84   86   88   90   92   94   96   98  100  102  104  106  108  110
  112  114  116  118  120  122  124  126  128  130  132  134  136  138
  140  142  144  146  148  150  152  154  156  158  160  162  164  166
  168  170  172  174  176  178  180  182  184  186  188  190  192  194
  196  198  200  202  204  206  208  210  212  214  216  218  220  222
  224  226  228  230  232  234  236  238  240  242  244  246  248  250
  252  254  256  258  260  262  264  266  268  270  272  274  276  278
  280  282  284  286  288  290  292  294  296  298  300  302  304  306
  308  310  312  314  316  318  320  322  324  326  328  330  332  334
  336  338  340  342  344  346  348  350  352  354  356  358  360  362
  364

In [ ]:
#  ./Experiments_vacancy/simulation_conc0.50-0.50_vacancy_0.1%.json


# './Experiments_vacancy/param_experiments_conc0.50-0.50_vacancy_0.1%.json'